## Задание 1

#### Простая задача, можно решить любым способом. Необходимо узнать является ли слово палиндромом без учета регистра (ШаЛАш - палиндром) и без учета пробелов (Ша Ла  ш - тоже палиндром). На вход подается строка, на выходе True или False.

In [1]:
word = input()

def palindrom(word):
    word = word.lower()
    s = ''.join(i for i in word if i.isalpha())
    print(s == s[::-1])
    
palindrom(word)



F88 77 88f
True


## Задание 2

#### Немного обработки данных. Необходимо открыть файл "Пример данных.csv". В нем пять признаков: Сегмент, Класс, Тип сделки, Цена и Описание. Реальные признаки только Описание и Тип сделки, остальные сгенерированы рандомно из заданного списка. Необходимо: <br><br>1) В сегментах есть пропуски, их необходимо заполнить исходя из имеющихся классов и соотношения классов и сегментов: К сегменту Торговая недвижимость относятся классы: [ТРЦ, РТЦ, Стрит-ритейл]; к сегменту Объекты свободного назначения относится класс [ОСН]; к сегменту Офисная недвижимость относятся классы [Офисы A, Офисы B, Офисы C]; к сегменту Производственно-складская нежвимость относятся классы:[ПС A, ПС B, ПС C]. На выходе во всех сегментах должен быть правильный класс. <br><br> 2) По каждому Сегменту + Классу + Типу сделки надо посчитать две статистики: среднюю цену и медиану цены. Всего таких статистик будет: Количество Сегментов * Количество Классов * Количество Типов сделки * 2 (средняя и медиана). <br><br>3) Необходимо заполнить признак "Отопление" (сейчас его нет). Он может принимать два значения - "Да" и "Нет". Необходимо вытащить его из Описания. Если Отопление есть в описании и не указано, что его нет, то считаем, что отопление есть (т.е. например "центральное отопление", "отопление есть" и т.д. = "Да", а если явно указано что его нет, то = "Нет"). Не обязательно все строки должны быть заполнены, где-то может и не быть никакой информации о том, есть ли отопление или его нет.<br><br>P.S. В этом задании мы хотим посмотреть как Вы умеете работать с данными и регулярными выражениями.

In [75]:
### Здесь и далее какой-то код

In [27]:
import pandas as pd
import numpy as np
import re

In [9]:
data = pd.read_csv(r'd:\temp\inform\task_1\data.csv')

In [10]:
data.head()

,Сегмент,Класс,Тип сделки,Описание,Цена
0,NaN,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,NaN,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22


In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
Сегмент       7951 non-null object
Класс         10000 non-null object
Тип сделки    10000 non-null object
Описание      10000 non-null object
Цена          10000 non-null int64
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [ ]:
#пробелы - плохо

In [12]:
data = data.rename(columns={"Тип сделки": "Тип_сделки"})

In [13]:
segment_type = {('ТРЦ', 'РТЦ', 'Стрит-ритейл') : 'Торговая недвижимость', 
                ('ОСН') : 'Объекты свободного назначения',
                ('Офисы A', 'Офисы B', 'Офисы C') : 'Офисная недвижимость', 
                ('ПС A', 'ПС B', 'ПС C'): 'Производственно-складская недвижимость'}
               
    


In [14]:
def segment_replace(a):
    l=[]
    for row in a:
        for i in segment_type.keys():
            if row in i:        
                l.append( segment_type[i])
    return pd.Series(l)           

In [15]:
data = data.replace({'Сегмент': 'Производственно-складская нежвимость'},'Производственно-складская недвижимость')

In [16]:
d = data.copy(deep=True)

In [18]:
data.head(10)

,Сегмент,Класс,Тип_сделки,Описание,Цена
0,NaN,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,NaN,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
5,Торговая недвижимость,РТЦ,Аренда,Ук викинг групп предлагает Вам в аренду от соб...,81
6,NaN,ПС C,Аренда,ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...,45
7,NaN,Офисы B,Аренда,"- 150 кв.м, ул. К.Назаровой. Склад или произв...",46
8,NaN,ПС C,Аренда,СДАМ в аренду теплое помещение: 500 м2 (в т.ч...,91
9,Производственно-складская недвижимость,ПС A,Продажа,Продается офисно-складской комплекс на ул. Тим...,83


In [19]:
data.loc[pd.isnull(data['Сегмент']),'Сегмент'] = segment_replace(data['Класс'])


In [20]:
data

,Сегмент,Класс,Тип_сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
...,...,...,...,...,...
9995,Офисная недвижимость,Офисы C,Продажа,Склад 1222кв.м. Отличное состояние. Два грузов...,28
9996,Торговая недвижимость,Стрит-ритейл,Продажа,"Лот:. Щелково г, Щелковское ш, 15 км от МКАД, ...",25
9997,Офисная недвижимость,Офисы C,Аренда,Часть производственно-складского помещения на ...,71
9998,Объекты свободного назначения,ОСН,Аренда,Прямая аренда от собственника.Предоставление ю...,69


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 5 columns):
Сегмент       10000 non-null object
Класс         10000 non-null object
Тип_сделки    10000 non-null object
Описание      10000 non-null object
Цена          10000 non-null int64
dtypes: int64(1), object(4)
memory usage: 390.8+ KB


In [22]:
data.groupby(['Сегмент', 'Класс', 'Тип_сделки']).agg(['mean', 'median'])

Цена  \
                                                                     mean   
Сегмент                                Класс        Тип_сделки              
Объекты свободного назначения          ОСН          Аренда      50.280952   
                                                    Продажа     50.921622   
Офисная недвижимость                   Офисы A      Аренда      50.829851   
                                                    Продажа     49.232143   
                                       Офисы B      Аренда      50.558559   
                                                    Продажа     51.718750   
                                       Офисы C      Аренда      50.667571   
                                                    Продажа     53.452308   
Производственно-складская недвижимость ПС A         Аренда      51.481250   
                                                    Продажа     48.729730   
                                       ПС B         Аренда      51.456456   
                                                    Продажа     50.118421   
                                       ПС C         Аренда      50.997956   
                                                    Продажа     48.556667   
Торговая недвижимость                  РТЦ          Аренда      47.296636   
                                                    Продажа     54.632911   
                                       Стрит-ритейл Аренда      51.970443   
                                                    Продажа     49.593123   
                                       ТРЦ          Аренда      50.153392   
                                                    Продажа     44.701493   

                                                                       
                                                               median  
Сегмент                                Класс        Тип_сделки         
Объекты свободного назначения          ОСН          Аренда         50  
                                                    Продажа        51  
Офисная недвижимость                   Офисы A      Аренда         50  
                                                    Продажа        52  
                                       Офисы B      Аренда         49  
                                                    Продажа        54  
                                       Офисы C      Аренда         51  
                                                    Продажа        56  
Производственно-складская недвижимость ПС A         Аренда         50  
                                                    Продажа        52  
                                       ПС B         Аренда         52  
                                                    Продажа        50  
                                       ПС C         Аренда         52  
                                                    Продажа        49  
Торговая недвижимость                  РТЦ          Аренда         46  
                                                    Продажа        56  
                                       Стрит-ритейл Аренда         53  
                                                    Продажа        48  
                                       ТРЦ          Аренда         51  
                                                    Продажа        41

In [23]:
data

,Сегмент,Класс,Тип_сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
...,...,...,...,...,...
9995,Офисная недвижимость,Офисы C,Продажа,Склад 1222кв.м. Отличное состояние. Два грузов...,28
9996,Торговая недвижимость,Стрит-ритейл,Продажа,"Лот:. Щелково г, Щелковское ш, 15 км от МКАД, ...",25
9997,Офисная недвижимость,Офисы C,Аренда,Часть производственно-складского помещения на ...,71
9998,Объекты свободного назначения,ОСН,Аренда,Прямая аренда от собственника.Предоставление ю...,69


In [24]:
data.loc[0:20,'Описание']

0     Id 10838в короткие сроки на безвозмездной осно...
1     Сдается объект на территории калининской базы....
2     Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...
3     Аренда склада. 3600 м2. Логистический комплекс...
4     Возможность подключения к системе отопления, з...
5     Ук викинг групп предлагает Вам в аренду от соб...
6     ОТ СОБСТВЕННИКА. Сдается в долгосрочную аренду...
7     - 150 кв.м, ул. К.Назаровой. Склад  или произв...
8     СДАМ в аренду  теплое помещение: 500 м2 (в т.ч...
9     Продается офисно-складской комплекс на ул. Тим...
10    Сдается металлический ангар, бетоннй пол,высот...
11    Id объекта в нашей базе: 4760 Сдам в аренду те...
12    Сдается помещение под склад в цоколе на ул. Пр...
13    Лот №277617 Без Комиссии от Собственника!🔥 Аре...
14    Аренда производственно-складского помещения на...
15    Лот:. Евгений. Помещение свободного назначения...
16    Id: 513605. Склад от собственника без комиссии...
17    Предлагаю арендовать помещение 250 кв.м. п

In [25]:
pd.options.display.max_colwidth = 1500

In [26]:
data.loc[40:50,'Описание']

40                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           Предлагаются в аренду от собственника помещения складского назначения.Класс: А. Расположение: Москва, с. Софьино, Киевская ул, д. 266.Метро: Жулебино.Подписание договора аренды с владельцем складского комплекса.Без комиссии. 0 переплат. Возможно обсуждение арендной ставки объекта.Арендная ставка указана с учетом ндсвыгодная транспортная доступность, выезд на магистрали города.

In [ ]:
б\отопления  
без отопления,   
нет 
без 
б\ 

In [ ]:
отапливаемое 
в отапливаемом 
отоплением
включено отопление 
Есть отопление 
Центральное отопление 
(Центральные коммуникации) 
отапливаемые склады
системами центрального отопления
возможно утеплить данное помещение и провести систему отопления

In [148]:
pd.options.display.max_colwidth = 50

In [139]:
pattern = r'((?:\bнет \b|\bбез \b|\bб.)отопления)'

In [142]:
def hot(s):
    
    match = re.findall(pattern, s,flags=re.IGNORECASE)
    if match:
        return ('Нет')
    else:
        return ('Да')

In [149]:
data

,Сегмент,Класс,Тип_сделки,Описание,Цена
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22
...,...,...,...,...,...
9995,Офисная недвижимость,Офисы C,Продажа,Склад 1222кв.м. Отличное состояние. Два грузов...,28
9996,Торговая недвижимость,Стрит-ритейл,Продажа,"Лот:. Щелково г, Щелковское ш, 15 км от МКАД, ...",25
9997,Офисная недвижимость,Офисы C,Аренда,Часть производственно-складского помещения на ...,71
9998,Объекты свободного назначения,ОСН,Аренда,Прямая аренда от собственника.Предоставление ю...,69


In [150]:
data['Отопление'] = [hot(x) for x in data['Описание']]

In [151]:
data

,Сегмент,Класс,Тип_сделки,Описание,Цена,Отопление
0,Офисная недвижимость,Офисы A,Аренда,Id 10838в короткие сроки на безвозмездной осно...,62,Да
1,Торговая недвижимость,Стрит-ритейл,Аренда,Сдается объект на территории калининской базы....,12,Да
2,Торговая недвижимость,ТРЦ,Аренда,"Тeлeфoния - Цифpa Oдин, Intinform; Коммунальны...",98,Да
3,Торговая недвижимость,Стрит-ритейл,Аренда,Аренда склада. 3600 м2. Логистический комплекс...,7,Да
4,Офисная недвижимость,Офисы C,Аренда,"Возможность подключения к системе отопления, з...",22,Да
...,...,...,...,...,...,...
9995,Офисная недвижимость,Офисы C,Продажа,Склад 1222кв.м. Отличное состояние. Два грузов...,28,Да
9996,Торговая недвижимость,Стрит-ритейл,Продажа,"Лот:. Щелково г, Щелковское ш, 15 км от МКАД, ...",25,Нет
9997,Офисная недвижимость,Офисы C,Аренда,Часть производственно-складского помещения на ...,71,Да
9998,Объекты свободного назначения,ОСН,Аренда,Прямая аренда от собственника.Предоставление ю...,69,Да


In [159]:
data.iloc[9995,3]

'Склад 1222кв.м. Отличное состояние. Два грузовых лифта. деборкадер. Офисный блок, две переговорные, зона рецепции, Все коммуникации: холодная, горячая вода, отопление. Приточно-вытяжная вентиляция. Сплинкерная система пожаротушения. На территории современного складского комплекса: охрана, пропускная система. Электоэнергия 128квт. Документы в порядке. Собственник юр. лицо.'